In [45]:
!pip install ipykernel --pre

In [14]:
import tarfile
import tempfile
import urllib.request
from pathlib import Path
from configparser import ConfigParser
import os

def get_data_url_from_model_zoo():
    url = 'https://raw.githubusercontent.com/NifTK/NiftyNetModelZoo/5-reorganising-with-lfs/highres3dnet_brain_parcellation/main.ini'
    with urllib.request.urlopen(url) as response:
        config_string = response.read().decode()
    config = ConfigParser()
    config.read_string(config_string)
    data_url = config['data']['url']
    return data_url


def download_data(data_url):
    # tempdir = Path(tempfile.gettempdir())
    tempdir = Path(os.getcwd())
    download_dir = tempdir / 'downloaded_data'
    download_dir.mkdir(exist_ok=True)
    data_path = download_dir / Path(data_url).name
    print(data_path)
    if not data_path.is_file():
        urllib.request.urlretrieve(data_url, data_path)
    with tarfile.open(data_path, 'r') as tar:
        tar.extractall(download_dir)
    nifti_files = download_dir.glob('**/*.nii.gz')
    return list(nifti_files)[0]


def test_infer():
    image_path = download_data(get_data_url_from_model_zoo())

test_infer()

/home2/dwnusa/workspace/cnnstudy/ch10_monai/downloaded_data/data.tar.gz


In [37]:
import nibabel as nib
import torch
import numpy as np
import SimpleITK as sitk

In [38]:
repo = 'fepegar/highresnet'
model_name = 'highres3dnet'
print(torch.hub.help(repo, model_name))
"HighRes3DNet by Li et al. 2017 for T1-MRI brain parcellation"
"pretrained (bool): load parameters from pretrained model"
model = torch.hub.load(repo, model_name, pretrained=True)

Using cache found in /home2/dwnusa/.cache/torch/hub/fepegar_highresnet_master



    HighRes3DNet by Li et al. 2017 for T1-MRI brain parcellation
    pretrained (bool): load parameters from pretrained model
    


Using cache found in /home2/dwnusa/.cache/torch/hub/fepegar_highresnet_master


In [39]:
def check_header(nifti_image):
    orientation = ''.join(nib.aff2axcodes(nifti_image.affine))
    spacing = nifti_image.header.get_zooms()
    one_iso = 1, 1, 1
    is_ras = orientation == 'RAS'
    if not is_ras:
        print(f'Detected orientation: {orientation}. Reorienting to RAS...')
    is_1_iso = np.allclose(spacing, one_iso)
    if not is_1_iso:
        print(f'Detected spacing: {spacing}. Resampling to 1 mm iso...')
    needs_resampling = not is_ras or not is_1_iso
    return needs_resampling

def resample_ras_1mm_iso(nifti, interpolation=None):
    if interpolation is None:
        interpolation = sitk.sitkLinear
    nii_ras = nib.as_closest_canonical(nifti)
    spacing = nii_ras.header.get_zooms()
    one_iso = 1, 1, 1
    if np.allclose(spacing, one_iso):
        return nii_ras
    nii_resampled = resample_spacing(
        nii_ras,
        output_spacing=one_iso,
        interpolation=interpolation,
    )
    return nii_resampled

In [41]:
input_path1 = "_data/gaain/AD01_MR.nii"
# nii1 = nib.load(str(input_path1))
input_path2 = "_data/OASIS/OAS1_MR.nii"
# nii2 = nib.load(str(input_path2))
# print(nii1)
# print(nii2)

In [44]:
input_path = input_path2
nii = nib.load(str(input_path))
needs_resampling = check_header(nii)
print(needs_resampling)

Detected orientation: ASL. Reorienting to RAS...
True


In [35]:
if needs_resampling:
    nii = resample_ras_1mm_iso(nii)
data = nii.get_fdata()

Detected orientation: LPS. Reorienting to RAS...


ValueError: operands could not be broadcast together with shapes (4,) (3,) 

In [29]:
data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.